In [ ]:
import numpy as np
import glob
import pandas as pd
import matplotlib.image as img
import matplotlib.pyplot as plt
from minpy.context import cpu, gpu

In [45]:
class Neural_Net:
    
    def __init__(self):
        self.layers = list()
        
    def add_layer(self, neurons, activation = 'sigmoid'):
        self.layers.append( Layer(neurons, activation = activation))
        
    def set_input(self, inp):
        self.inp = inp
        self.train_inp = inp
        
        
    def set_output(self, out):
        self.out = out
        self.train_out = out
        
    def feed_forward(self):
        self.layers[0].set_input(self.train_inp)
        for i in range(1, len(self.layers) ):
            (self.layers[i]).set_input(self.layers[i-1].output)
        
        self.diff = self.layers[-1].output - self.train_out
        
    def back_propagation(self, lr):
        
        tensor = list()
        tensor.append(self.diff)
        for i in reversed(range(0, len(self.layers) - 1)):
            
            tensor.append(np.multiply(
                np.matmul(tensor[-1], np.transpose(self.layers[i+1].weights)),
                self.layers[i].d_output
                ))
        
        tensor = tensor[::-1]
        for i in range(len(tensor)):
            self.layers[i].weights -= lr*np.matmul(np.transpose(self.layers[i].inp), tensor[i])
            self.layers[i].bias -= lr*np.matmul(np.transpose(self.layers[i].A), tensor[i])
        
    def train(self, epochs, lr = 1, batch_size = None):
        if batch_size is None:
            batch_size = self.inp.shape[0]
        partitions = int(self.inp.shape[0]/batch_size)
        limits = np.linspace(0, self.inp.shape[0], partitions + 1, dtype= int)
        
            
        for epoch in range(epochs):
            print('epoch = ', epoch)
            for partition in range(partitions):
                
                
                self.train_inp = self.inp[int(limits[partition]):int(limits[partition + 1])]
                self.train_out = self.out[int(limits[partition]):int(limits[partition + 1])]
                
                self.feed_forward()
                self.back_propagation(lr)
            
            self.train_inp = self.inp
            self.train_out = self.out
            self.feed_forward()
            self.cost = np.sum((self.diff**2))
            print(self.cost)
    
        

In [46]:
class Layer:
    
    @staticmethod
    def sigmoid(x):
        return 1/(1+np.exp(-x))
    
    @staticmethod
    def relu(x):
        
        return np.where(x < 0, 0, x )
    
    @staticmethod
    def d_sigmoid(x):
        a = Layer.sigmoid(x)
        return a*(1-a)
    
    @staticmethod
    def d_relu(x):
        
        return np.where(x<0, 0, 1)
    
    function = { 'sigmoid': (lambda x: Layer.sigmoid(x)), 'relu': (lambda x: Layer.relu(x))}
    
    d_function = { 'sigmoid': (lambda x: Layer.d_sigmoid(x)), 'relu': (lambda x: Layer.d_relu(x)) }


    def __init__(self, neurons, inp = None, activation = 'sigmoid'):
        self.init = False
        self.neurons = neurons
        self.inp = self.set_input(inp)
        self.activation = activation
        
        
    def set_input(self, inp):
        if (inp is None):
            return None
        
        if (self.init == False):
            self.weights = np.random.rand(inp.shape[1], self.neurons)
            print (self.weights)
            self.bias = np.random.rand(1, self.neurons)
            
            self.init = True
            
        self.inp = inp
        self.A = np.ones((inp.shape[0], 1))
        self.linear = np.add(
                        np.matmul(self.inp, self.weights), 
                        np.matmul(self.A, self.bias)
                        )
        self.output = Layer.function[self.activation](self.linear)
        self.d_output = Layer.d_function[self.activation](self.linear)

In [47]:
a = Neural_Net()

In [48]:
a.add_layer(100, activation = 'sigmoid')
a.add_layer(1, activation = 'sigmoid')

In [49]:
inp = np.array([[0, 0], [0, 1], [1,0], [1,1]])
inp[0:1]

array([[0., 0.]], dtype=float32)

In [50]:
out = np.array([[0], [1], [1], [1]])

In [51]:
a.set_input(inp)
a.set_output(out)

In [52]:
a.train(epochs = 10, lr = 1e-3)

epoch =  0
[[0.55559431 0.2557361  0.45052085 0.11534155 0.896896   0.27847232
  0.66468908 0.96120398 0.92320827 0.30977793 0.10768642 0.39618304
  0.16969142 0.71843216 0.17225959 0.53258711 0.96771477 0.83502614
  0.43094024 0.36313038 0.60004498 0.93370521 0.13860886 0.72084957
  0.78646978 0.43884704 0.5271726  0.16841588 0.17156733 0.79034448
  0.1875565  0.93920903 0.75929538 0.36226916 0.44369825 0.09130094
  0.27223379 0.97144399 0.95507091 0.96257899 0.02742183 0.35839412
  0.12554456 0.18463367 0.90954181 0.92588286 0.12137741 0.72894699
  0.19054148 0.54792485 0.81381627 0.94561406 0.57753531 0.92721634
  0.89575524 0.50572004 0.59116852 0.37606367 0.07676039 0.60361218
  0.9725237  0.98514528 0.80024752 0.66938648 0.21512035 0.63712205
  0.03687289 0.89403523 0.60444921 0.93322675 0.13281692 0.29181224
  0.04532149 0.81300427 0.96482535 0.64190831 0.65690932 0.47824314
  0.49222237 0.51504806 0.24310962 0.68313423 0.24705421 0.71235669
  0.07682565 0.60851035 0.91555432 0.

In [ ]:
a.layers[0].weights

In [ ]:
a.layers[1].output

In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist



In [ ]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train.reshape(-1, 28*28) / 255.0, x_test.reshape(-1, 28*28) / 255.0
y_train = np.transpose(np.array([np.where(y_train == element, 1, 0) for element in np.unique(y_train)]))
y_test = np.transpose(np.array([np.where(y_test == element, 1, 0) for element in np.unique(y_test)]))

In [ ]:
y_train[1]

In [ ]:
plt.imshow(x_train[1].reshape(28, 28))

In [ ]:
import numpy as np
import time
model = Neural_Net()
model.add_layer(10, activation = 'sigmoid')
model.set_input(x_train)
model.set_output(y_train)
t0 = time.time()
model.train(lr = 1e-5, epochs = 2, batch_size = None)
t1 = time.time()

In [ ]:
import minpy.numpy as np
from minpy.context import cpu, gpu
with gpu(0):
    model = Neural_Net()
    model.add_layer(10, activation = 'sigmoid')
    model.set_input(np.array(x_train))
    model.set_output(np.array(y_train))
    t2 = time.time()
    model.train(lr = 1e-5, epochs = 2, batch_size = None)
    t3 = time.time()

In [ ]:
print("run on cpu: %.6f s/iter" % ((t1 - t0) / n))
print("run on gpu: %.6f s/iter" % ((t3 - t2) / n))

In [ ]:
model.train(lr = 1e-5, epochs = 2, batch_size = 1000)

In [ ]:
model.set_input(x_train)
model.set_output(y_train)
model.feed_forward()
predict = np.argmax(model.layers[-1].output, axis=1)
y = np.argmax(y_train, axis=1)
accuracy = (y == predict)
score = sum(np.where(accuracy == True, 1, 0))/len(y)
print('Test Accuracy =', score)

In [ ]:
model.set_input(x_test)
model.set_output(y_test)
model.feed_forward()
predict = np.argmax(model.layers[-1].output, axis=1)
y = np.argmax(y_test, axis=1)
accuracy = (y == predict)
score = sum(np.where(accuracy == True, 1, 0))/len(y)
print('Test Accuracy =', score)